Для решения данной краевой задачи методом переменных направлений, необходимо разбить область на прямоугольную сетку и дискретизировать уравнение разностным аналогом.

Будем использовать следующие обозначения: $u_{i,j,k}$ соответствует значению функции $u$ на сетке в точке $(x_i, y_j, t_k)$, $h$ - шаг сетки по $x$ и $y$, $\tau$ - шаг сетки по времени.

Тогда уравнение разностного аналога можно записать следующим образом:

$$
    \frac{u_{i,j,k+1} - u_{i,j,k}}{\tau} = \frac{1}{h^2} (u_{i+1,j,k} - 2u_{i,j,k} + u_{i-1,j,k} + u_{i,j+1,k} - 2u_{i,j,k} + u_{i,j-1,k}) + \sin (\pi x_i) \cdot y_j(y_j-1) t_k
$$

Перенесем все неизвестные на левую сторону, а известные на правую:

$$
    -\frac{\tau}{h^2} u_{i-1,j,k} + \left(1 + \frac{2\tau}{h^2}\right)u_{i,j,k} - \frac{\tau}{h^2}u_{i+1,j,k} - \frac{\tau}{h^2}u_{i,j-1,k} + \left(1 + \frac{2\tau}{h^2}\right)u_{i,j,k} - \frac{\tau}{h^2}u_{i,j+1,k} - \sin (\pi x_i) \cdot y_j(y_j-1) \tau t_k = u_{i,j,k+1}
$$

Введем следующие обозначения:

$$
    a_{i,j} = -\frac{\tau}{h^2} \\
    b_{i,j} = 1 + \frac{2\tau}{h^2} \\
    c_{i,j} = -\frac{\tau}{h^2} \\
    d_{i,j,k} = -\sin (\pi x_i) \cdot y_j(y_j-1) \tau t_k
$$

Тогда получаем систему линейных уравнений с трехдиагональной матрицей:

$$
    a_{i,j}u_{i-1,j,k} + b_{i,j}u_{i,j,k} + c_{i,j}u_{i+1,j,k} + a_{i,j}u_{i,j-1,k} + b_{i,j}u_{i,j,k} + c_{i,j}u_{i,j+1,k} + d_{i,j,k} = u_{i,j,k+1}
$$

Для решения этой системы применим метод прогонки. Решение будем искать на каждом шаге по времени.

Теперь реализуем алгоритм в коде на Python. 

Сначала зададим начальные значения, размер сетки и шаг по времени:

```python
import numpy as np
import matplotlib.pyplot as plt

# Определяем начальные значения функции и параметры сетки
N = 20
h = 1/N
tau = 0.1/N**2

# Инициализируем сетку
u = np.zeros((N+1,N+1,int(1/tau)+1))
```

Теперь заполним начальное и граничные значения:

```python
# Начальное значение
u[:,:,0] = 0

# Нулевые граничные условия
u[0,:,:] = 0
u[N,:,:] = 0
u[:,0,:] = 0
u[:,N,:] = 0
```

Следующим шагом будет реализация метода прогонки. Для этого напишем функцию solve_tridiagonal:

```python
def solve_tridiagonal(a, b, c, d):
    alpha = np.zeros(len(d)-1)
    beta = np.zeros(len(d)-1)
    x = np.zeros(len(d))

    alpha[0] = -c[0]/b[0]
    beta[0] = d[0]/b[0]

    for i in range(1, len(d)-1):
        alpha[i] = -c[i]/(a[i]*alpha[i-1]+b[i])
        beta[i] = (d[i]-a[i]*beta[i-1])/(a[i]*alpha[i-1]+b[i])

    x[-1] = (d[-1]-a[-1]*beta[-1])/(a[-1]*alpha[-1]+b[-1])

    for i in range(len(d)-2, -1, -1):
        x[i] = alpha[i]*x[i+1]+beta[i]

    return x
```

Теперь можем реализовать алгоритм метода переменных направлений:

```python
# Цикл по времени
t = 0
while t < 1:
    t += tau
    # Шаг по x
    for j in range(1, N):
        a = np.zeros(N-1)
        b = np.zeros(N-1)
        c = np.zeros(N-1)
        d = np.zeros(N-1)

        for i in range(1, N):
            a[i-1] = -tau/h**2
            b[i-1] = 1 + 2*tau/h**2
            c[i-1] = -tau/h**2
            d[i-1] = -np.sin(np.pi*i*h)*j*h*(j*h-1)*tau*t + u[i,j,t//tau]

        x = solve_tridiagonal(a, b, c, d)

        for i in range(1, N):
            u[i,j,t//tau+1] = x[i-1]

    # Шаг по y
    for i in range(1, N):
        a = np.zeros(N-1)
        b = np.zeros(N-1)
        c = np.zeros(N-1)
        d = np.zeros(N-1)

        for j in range(1, N):
            a[j-1] = -tau/h**2
            b[j-1] = 1 + 2*tau/h**2
            c[j-1] = -tau/h**2
            d[j-1] = -np.sin(np.pi*i*h)*j*h*(j*h-1)*tau*t + u[i,j,t//tau+1]

        x = solve_tridiagonal(a, b, c, d)

        for j in range(1, N):
            u[i,j,t//tau+1] = x[j-1]
```

Теперь можем вывести результаты в виде графика и анимации:

```python
# График функции в последний момент времени
plt.imshow(u[:,:,int(1/tau)], cmap='jet', origin='lower', extent=[0,1,0,1])
plt.colorbar()
plt.title('Solution at t=1')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# Анимация изменения функции во времени
fig = plt.figure()
im = plt.imshow(u[:,:,0], cmap='jet', origin='lower', extent=[0,1,0,1])
plt.colorbar()
plt.title('Solution at t=0')
plt.xlabel('x')
plt.ylabel('y')
plt.tight_layout()

def update_fig(t):
    im.set_data(u[:,:,t])
    plt.title('Solution at t=' + str(round(t*tau,2)))
    return im,

ani = animation.FuncAnimation(fig, update_fig, frames=int(1/tau)+1, interval=50, blit=True)
ani.save('solution.mp4', writer=animation.FFMpegWriter(fps=25))
```